# import

In [14]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [15]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [16]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.softmax_cross_entropy_with_logits_v2(y_pred = logits, y_true = labels,
                                                                        from_logits = True))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [1]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm(rate):
    return tf.keras.layers.BatchNormalization()

# Create model(class version)

In [19]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.RandomNormal() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())
            # 일반적으로 layer-norm-activation function 순서임.
            # norm-activation function-layer 순서로 하는 경우도 있음.
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [2]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(batch_norm())
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [21]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [22]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [23]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 1.4222, train_loss: 2.30429530, train_accuracy: 0.2891, test_accuracy: 0.1815
Epoch: [ 0] [    1/  468] time: 1.9150, train_loss: 2.29759312, train_accuracy: 0.2109, test_accuracy: 0.1167
Epoch: [ 0] [    2/  468] time: 2.4525, train_loss: 2.36467552, train_accuracy: 0.1562, test_accuracy: 0.1496
Epoch: [ 0] [    3/  468] time: 3.0070, train_loss: 2.24743557, train_accuracy: 0.2266, test_accuracy: 0.1929
Epoch: [ 0] [    4/  468] time: 3.5266, train_loss: 2.34653759, train_accuracy: 0.1953, test_accuracy: 0.2002
Epoch: [ 0] [    5/  468] time: 4.0173, train_loss: 2.29403114, train_accuracy: 0.2734, test_accuracy: 0.2510
Epoch: [ 0] [    6/  468] time: 4.5709, train_loss: 2.29108858, train_accuracy: 0.2891, test_accuracy: 0.3323
Epoch: [ 0] [    7/  468] time: 5.0316, train_loss: 2.24359846, train_accuracy: 0.4297, test_accuracy: 0.4281
Epoch: [ 0] [    8/  468] time: 5.4953, 

Epoch: [ 0] [   74/  468] time: 34.9830, train_loss: 0.72964191, train_accuracy: 0.8906, test_accuracy: 0.8822
Epoch: [ 0] [   75/  468] time: 35.3909, train_loss: 0.77122891, train_accuracy: 0.8750, test_accuracy: 0.8827
Epoch: [ 0] [   76/  468] time: 35.7716, train_loss: 0.66962230, train_accuracy: 0.8984, test_accuracy: 0.8810
Epoch: [ 0] [   77/  468] time: 36.2236, train_loss: 0.42279506, train_accuracy: 0.9453, test_accuracy: 0.8838
Epoch: [ 0] [   78/  468] time: 36.6684, train_loss: 0.50701046, train_accuracy: 0.9141, test_accuracy: 0.8814
Epoch: [ 0] [   79/  468] time: 37.1282, train_loss: 0.58169210, train_accuracy: 0.9219, test_accuracy: 0.8747
Epoch: [ 0] [   80/  468] time: 37.5838, train_loss: 0.56643480, train_accuracy: 0.9062, test_accuracy: 0.8732
Epoch: [ 0] [   81/  468] time: 38.0404, train_loss: 0.64585388, train_accuracy: 0.8984, test_accuracy: 0.8737
Epoch: [ 0] [   82/  468] time: 38.4973, train_loss: 0.60963511, train_accuracy: 0.8906, test_accuracy: 0.8750
E

Epoch: [ 0] [  148/  468] time: 68.4232, train_loss: 0.32161719, train_accuracy: 0.9375, test_accuracy: 0.9242
Epoch: [ 0] [  149/  468] time: 68.8282, train_loss: 0.41402996, train_accuracy: 0.9219, test_accuracy: 0.9241
Epoch: [ 0] [  150/  468] time: 69.2971, train_loss: 0.36431974, train_accuracy: 0.9219, test_accuracy: 0.9240
Epoch: [ 0] [  151/  468] time: 69.7559, train_loss: 0.36320797, train_accuracy: 0.9609, test_accuracy: 0.9237
Epoch: [ 0] [  152/  468] time: 70.2107, train_loss: 0.42218721, train_accuracy: 0.9375, test_accuracy: 0.9232
Epoch: [ 0] [  153/  468] time: 70.6724, train_loss: 0.57954276, train_accuracy: 0.8906, test_accuracy: 0.9219
Epoch: [ 0] [  154/  468] time: 71.1382, train_loss: 0.35657695, train_accuracy: 0.9219, test_accuracy: 0.9197
Epoch: [ 0] [  155/  468] time: 71.6099, train_loss: 0.36398834, train_accuracy: 0.9141, test_accuracy: 0.9182
Epoch: [ 0] [  156/  468] time: 72.0148, train_loss: 0.29787117, train_accuracy: 0.9531, test_accuracy: 0.9178
E

Epoch: [ 0] [  222/  468] time: 101.3029, train_loss: 0.30318385, train_accuracy: 0.9531, test_accuracy: 0.9380
Epoch: [ 0] [  223/  468] time: 101.6890, train_loss: 0.38466734, train_accuracy: 0.9375, test_accuracy: 0.9365
Epoch: [ 0] [  224/  468] time: 102.1406, train_loss: 0.41763908, train_accuracy: 0.8750, test_accuracy: 0.9361
Epoch: [ 0] [  225/  468] time: 102.6353, train_loss: 0.32467800, train_accuracy: 0.9297, test_accuracy: 0.9352
Epoch: [ 0] [  226/  468] time: 103.0890, train_loss: 0.27275658, train_accuracy: 0.9609, test_accuracy: 0.9358
Epoch: [ 0] [  227/  468] time: 103.5401, train_loss: 0.36735442, train_accuracy: 0.9453, test_accuracy: 0.9356
Epoch: [ 0] [  228/  468] time: 103.9938, train_loss: 0.34415174, train_accuracy: 0.9531, test_accuracy: 0.9356
Epoch: [ 0] [  229/  468] time: 104.4367, train_loss: 0.29496324, train_accuracy: 0.9688, test_accuracy: 0.9354
Epoch: [ 0] [  230/  468] time: 104.8707, train_loss: 0.25996780, train_accuracy: 0.9766, test_accuracy:

Epoch: [ 0] [  296/  468] time: 134.2325, train_loss: 0.34261495, train_accuracy: 0.9531, test_accuracy: 0.9444
Epoch: [ 0] [  297/  468] time: 134.6434, train_loss: 0.35213327, train_accuracy: 0.9609, test_accuracy: 0.9431
Epoch: [ 0] [  298/  468] time: 135.0892, train_loss: 0.21382579, train_accuracy: 0.9531, test_accuracy: 0.9437
Epoch: [ 0] [  299/  468] time: 135.5350, train_loss: 0.37368944, train_accuracy: 0.9297, test_accuracy: 0.9439
Epoch: [ 0] [  300/  468] time: 135.9918, train_loss: 0.26881146, train_accuracy: 0.9453, test_accuracy: 0.9446
Epoch: [ 0] [  301/  468] time: 136.4366, train_loss: 0.23571074, train_accuracy: 0.9688, test_accuracy: 0.9445
Epoch: [ 0] [  302/  468] time: 136.8834, train_loss: 0.23484790, train_accuracy: 0.9609, test_accuracy: 0.9447
Epoch: [ 0] [  303/  468] time: 137.3322, train_loss: 0.17869462, train_accuracy: 0.9766, test_accuracy: 0.9447
Epoch: [ 0] [  304/  468] time: 137.7790, train_loss: 0.45602924, train_accuracy: 0.8984, test_accuracy:

Epoch: [ 0] [  370/  468] time: 168.0036, train_loss: 0.25469375, train_accuracy: 0.9453, test_accuracy: 0.9440
Epoch: [ 0] [  371/  468] time: 168.5232, train_loss: 0.35663542, train_accuracy: 0.9219, test_accuracy: 0.9442
Epoch: [ 0] [  372/  468] time: 169.0697, train_loss: 0.21490657, train_accuracy: 0.9609, test_accuracy: 0.9458
Epoch: [ 0] [  373/  468] time: 169.5628, train_loss: 0.21961062, train_accuracy: 0.9453, test_accuracy: 0.9470
Epoch: [ 0] [  374/  468] time: 170.0415, train_loss: 0.35540432, train_accuracy: 0.9375, test_accuracy: 0.9495
Epoch: [ 0] [  375/  468] time: 170.5393, train_loss: 0.42924726, train_accuracy: 0.9531, test_accuracy: 0.9503
Epoch: [ 0] [  376/  468] time: 171.0320, train_loss: 0.16703720, train_accuracy: 0.9609, test_accuracy: 0.9518
Epoch: [ 0] [  377/  468] time: 171.5268, train_loss: 0.34285867, train_accuracy: 0.9531, test_accuracy: 0.9533
Epoch: [ 0] [  378/  468] time: 172.0574, train_loss: 0.47525984, train_accuracy: 0.8984, test_accuracy:

Epoch: [ 0] [  444/  468] time: 203.9896, train_loss: 0.21695200, train_accuracy: 0.9297, test_accuracy: 0.9527
Epoch: [ 0] [  445/  468] time: 204.4474, train_loss: 0.20704252, train_accuracy: 0.9219, test_accuracy: 0.9540
Epoch: [ 0] [  446/  468] time: 204.9052, train_loss: 0.38743705, train_accuracy: 0.9531, test_accuracy: 0.9545
Epoch: [ 0] [  447/  468] time: 205.3869, train_loss: 0.18031266, train_accuracy: 0.9453, test_accuracy: 0.9540
Epoch: [ 0] [  448/  468] time: 205.8527, train_loss: 0.25061899, train_accuracy: 0.9375, test_accuracy: 0.9550
Epoch: [ 0] [  449/  468] time: 206.3045, train_loss: 0.23444165, train_accuracy: 0.9766, test_accuracy: 0.9552
Epoch: [ 0] [  450/  468] time: 206.7114, train_loss: 0.31601828, train_accuracy: 0.9297, test_accuracy: 0.9569
Epoch: [ 0] [  451/  468] time: 207.1462, train_loss: 0.23507050, train_accuracy: 0.9609, test_accuracy: 0.9564
Epoch: [ 0] [  452/  468] time: 207.6060, train_loss: 0.14901662, train_accuracy: 0.9531, test_accuracy:

In [24]:
# test acc = 95.63%